In [2]:
import sys

sys.path.append("..")
%load_ext autoreload
%autoreload 2
from trl_wrapper.trainer_wrapper import TrainerWrapper, GRPO_WRITING_CONFIG, SMOL_LM_135M

cfg = GRPO_WRITING_CONFIG
cfg.train_batch_size = 2
cfg.num_generations = 2
cfg.model_id_or_path = SMOL_LM_135M
cfg.notebook_mode = True
cfg.max_samples = 1000

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
wrapper = TrainerWrapper(cfg)
wrapper.init_model()

2025-04-01 20:10:59.950 | INFO     | trl_wrapper.trainer_wrapper:__init__:285 - Setting padding side to: left
2025-04-01 20:10:59.963 | INFO     | trl_wrapper.trainer_wrapper:init_model:296 - Using device: mps
2025-04-01 20:10:59.963 | INFO     | trl_wrapper.trainer_wrapper:init_model:310 - Loading model HuggingFaceTB/SmolLM2-135M-Instruct with attn_impl: sdpa


In [10]:
wrapper.init_data_module()

2025-04-01 20:15:23.133 | INFO     | trl_wrapper.trainer_wrapper:init_data_module:336 - Using chat template override: smollmv2
2025-04-01 20:15:23.133 | INFO     | trl_wrapper.wrapper_config:__init__:133 - Cache dir: ../dataset_caches/writing_g_r_p_o_data_module


In [ ]:
wrapper.data_module.train_dataset[0]

In [ ]:
wrapper.init_trainer()

In [ ]:
from model.reasoning import GSM8K_SYSTEM_PROMPT, CONNECTIONS_PROMPT
from model.utils import get_available_device

messages = [
    {"role": "system", "content": CONNECTIONS_PROMPT},
    {
        "role": "user",
        "content": "train, panda, dove, series, wind, bear, orca, bass, string, skunk, speed, sand, zebra, tourist, desert, chain",
    },
]
tokenized_chat = wrapper.tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
    return_attention_mask=True,
)

device = get_available_device()
tokenized_chat = tokenized_chat.to(device)
out = wrapper.model.generate(tokenized_chat, max_length=1024)
print(wrapper.tokenizer.decode(out[0], skip_special_tokens=True))

In [ ]:
wrapper.train()